<a href="https://colab.research.google.com/github/matech96/DeepLearningAVizualisInformatikaban/blob/master/HF1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Learning Alkalmazása a Vizuális Informatikában 
##1. Házi feladat

**Fontos tudnivalók**

*   A házi feladatot önállóan kell elvégezni. Teljes megoldások máshonnan történő másolása nem elfogadható.
*   Ez a notebook egyben megoldás és dokumentáció. Az eredményeket értékeljétek ki, printelgessetek, ahhoz fűzzetek szépen megszerkesztett kommentárokat, stb.
*   Egy rész feladatot nem kell egyetlen kód blokkban megoldani. Sokkal szebb és struktúráltab lesz a munka, ha logikailag összetartozó egységekre bontjátok és azokat szépen egyesével magyarázzátok.
*   A házi feladat elkészítésére 3 hét áll rendelkezésetekre. Beadni az Edu portálon lehet. A határidő Március 12. 12:00.

### 1. Rész

Implementálj PyTorch-ban 2D konvolúciós réteget az NN, valamint a Functional modulok használata nélkül. A réteg konstruktorának a következő bemeneti paraméterei legyenek:


*   Kernel méret (n)
*   Bemeneti csatornaszám (inCH)
*   Kimeneti csatornaszám (oCH)

A rétegnek legyen forward függvénye, amelyben megkap egy [inCH x H x W] méretű tenzort, ahol H és W nem ismertek előre. A kimenete pedig legyen [oCH x H x W] (használj a kernel mérethez megfelelő paddinget).



###2. Rész

Amennyiben az implementáció elkészült, ellenőrizd az implementáció helyességét a PyTorch Conv2d modulja segítségével! (Tipp: Használj többfajta konfigurációt!)
https://pytorch.org/docs/stable/nn.html#conv2d

###3. Rész

Hozz létre egy egy bemeneti csatornás konvolúciós réteget, amelynek az egyes kimeneti csatornákhoz tartozó szűrőinek súlyait inicializáld véletlenszerűen. (A kimeneti csatornák száma legyen mondjuk 3.)

Ezt követően olvass be egy képet, szürkeárnyalatosítsd, és futtass le OpenCV segítségével 3 általad választott konvolúciós szűrőt!

Végezetül tanítsd be arra a létrehozott konvolúciós réteget, hogy az OpenCV-s szűréseket minél pontosabban reprodukálni tudja. Vizsgáld meg az eredményként kapott szűrőket! (Tipp: Az OpenCV-vel szűrt képeket használd referenciaként, mondjuk a négyzetes hiba költséggel. Több képet egy tömbbe összepakolni np.stack segítségével lehet.)